In [1]:
## Loading Config Files

import os 
import json
import sys
sys.path.append("..")

with open('../config/runner_config.json') as json_file:
    json_data = json.load(json_file)


In [ ]:
## Data Aggreagator Run
from deployment.Data_Retreiver import Data_Retreiver
from deployment.Data_Aggregator import Aggregator

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

sql_table = json_data["sql_aggregate"]
sql_table_raw = json_data["sql_raw_energy"]


# Create Threads for All the devices
aggre = Aggregator(devices,sql_table,sql_table_raw,sql_addr,sql_port, sql_user, sql_pw, sql_db)

#Do step and get the final output that would be printed to the log
aggre.do_step()
print(aggre.getLatest())

In [3]:
## Conctrol Enactor 
from deployment.Control_Enactor import Enactor


devices = json_data["devices"]

sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

# Run Examples
print("Retreive - Set Points")
print(enact.retreive_previous_plan(list(devices.keys())[1]))
print(enact.retreive_previous_plan(list(devices.keys())[2]))
print("Enact Set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Same set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Diffrent set points")
print(enact.enact_light_plan(list(devices.keys())[1], 4320, 4320))
print(enact.enact_socket_plan(list(devices.keys())[9], 1200000))


Retreive - Set Points
(4320, 4320)
Previous File not There
None
Enact Set points
Making Changes on Website for:  Streetlight_No_1
True
Making Changes on Website for:  Playground_AC_socket_No_1
True
Enact Same set points
False
False
Enact Diffrent set points
Making Changes on Website for:  Streetlight_No_1
True
Making Changes on Website for:  Playground_AC_socket_No_1
True


In [6]:
#Data retreiver Run
import datetime
import pandas as pd 
from deployment.Data_Retreiver import Data_Retreiver
devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

print("--Retreive filled data--\n",data.get_unsent_logs(datetime.datetime.now()))
print("--Retreive filled data--\n",data.retreive_filled_aggre(datetime.datetime.now() - datetime.timedelta(days=20), 2))

dev_light = "Nursery_1A_CPE_No_2"
dev_socket = "Playground_AC_socket_No_1"
group = ["Nursery_1A_CPE_No_1", "Nursery_1A_CPE_No_2", "Nursery_1B_CPE_No_3", "Nursery_1B_CPE_No_4",
         "Nursery_1C_CPE_No_5", "Nursery_1C_CPE_No_6"]
hour = 6

pd.set_option('display.max_columns', 30)
dt_ts = datetime.datetime.now()
print("--Forecast--\n", data.retreive_latest_forecast(dt_ts).head(5))
print("--Raw Sys--\n", data.retreive_latest_raw_system_snapshot(dt_ts).head(5))
print("--Priority--\n", data.retreive_latest_priority(dt_ts).head(5))
print("--Group Energy--\n", data.get_total_energy_for_group(group, dt_ts))
print("--Aggregated Values --\n", data.retreive_aggregared_values(dt_ts))
print("--AVG for Hour Values --\n", data.retreive_average_vals_for_hour(dt_ts, hour,8))
print("--Average P Light--\n", data.retreive_average_P_lights(dev_light, dt_ts))
print("--AC Session--\n", data.retreive_AC_Session(dev_socket, dt_ts))
print("--AC Energy--\n", data.retreive_AC_Energy(dev_socket, dt_ts))
print("--Light Session--\n", data.retreive_Light_Session(dev_light, dt_ts))
print("--Light Energy--\n", data.retreive_Light_Energy(dev_light, dt_ts))
print("--Quota Latest Light--\n", data.get_latest_quota(dev_light, dt_ts))
print("--Quota Latest Socket --\n", data.get_latest_quota(dev_socket, dt_ts))



--Retreive filled data--
    id                        timestamp  transmitted  message_id   mem  cpu  \
0   2 2020-06-09 12:41:16.012093+00:00        False         1.0  79.7  5.7   
1   3 2020-06-09 12:41:53.568862+00:00        False         1.0  79.9  7.9   

   storage_perc  storage_free  temperature  user_actions  hist_system_load  \
0          43.5     262.89820          0.0          18.0           276.526   
1          43.5     262.89795          0.0          18.0           276.526   

   hist_consumed_energy  hist_generated_energy  hist_streetlight_no_1  \
0                294.25                1109.75                    0.0   
1                294.25                1109.75                    0.0   

   hist_streetlight_no_2             ...               \
0                    0.0             ...                
1                    0.0             ...                

   quota2_playground_ac_socket_no_1  quota1_playground_ac_socket_no_2  \
0                               NaN    

--AC Session--
 4709.0
--AC Energy--
 0.0
--Light Session--
 (None, 0.0)
--Light Energy--
 (0.0, 0.0)
--Quota Latest Light--
 (4329.0, 4320.0)
--Quota Latest Socket --
 (1200000.0, None)


In [7]:
from deployment.Battery import Battery

bat = Battery(state_of_charge=94.0, battery_capacity=18.79, max_discharge=40.0, charge_efficiency=0.85)
print(bat)
print("---Storage Capacity---\n", bat.get_storage_capacity_left())
print("---Discharge Capacity---\n", bat.get_discharge_capacity_left())
print("---Discharge Battery---\n", bat.discharge_battery(1.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(0.4))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)


SoC: 94.0 Energy_Left: 10.1466
---Storage Capacity---
 1.3263529411764705
---Discharge Capacity---
 10.1466
---Discharge Battery---
 0.0
SoC: 83.88823842469398 Energy_Left: 8.246599999999999
---Charge Battery---
 0.0
SoC: 85.6977115486961 Energy_Left: 8.586599999999999
---Discharge Battery---
 0.0
SoC: 70.26397019691323 Energy_Left: 5.686599999999997
---Discharge Battery---
 0.0
SoC: 54.83022884513037 Energy_Left: 2.786599999999997
---Discharge Battery---
 0.11340000000000305
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Charge Battery---
 0.0
SoC: 59.90420436402342 Energy_Left: 3.7400000000000007
---Charge Battery---
 0.0
SoC: 79.80840872804684 Energy_Left: 7.480000000000001
---Charge Battery---
 0.0
SoC: 99.71261309207026 Energy_Left: 11.22
---Charge Battery---
 4.336470588235296
SoC: 100.0 Energy_Left: 11.274
---Charge Battery---
 4.4
SoC: 100.0 Energy_Left: 11.274


In [11]:
# Controller Run

#Data retreiver Run
import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Control_Enactor import Enactor
from deployment.Controller import Controller

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

## Conctrol Enactor 
sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

#Controller Init
ts_step = datetime.datetime.now()  # - datetime.timedelta(hours=3)

allocation = json_data["allocation"]
if 'System_Data' in allocation:
    del allocation['System_Data']

control = Controller(data, enact, allocation, 4)
print(control.getLatest())

control.do_step(ts_step)
print(control.getLatest())

Controller : Initilised
Controller : Reverting to standard setup as no data is available
Decisions: {'nursery1_lights': {'state': 'Unconstrained', 'energy_est_used_total': 0, 'constraining_factor': 1.0, 'device_const': {'Nursery_1A_CPE_No_1': (4329, 4320, False), 'Nursery_1A_CPE_No_2': (4329, 4320, False), 'Nursery_1B_CPE_No_3': (4329, 4320, False), 'Nursery_1B_CPE_No_4': (4329, 4320, False), 'Nursery_1C_CPE_No_5': (4329, 4320, False), 'Nursery_1C_CPE_No_6': (4329, 4320, False)}, 'timestamp': datetime.datetime(2020, 6, 9, 13, 23, 25, 358279)}, 'nursery1_sockets': {'state': 'Unconstrained', 'energy_est_used_total': 0, 'constraining_factor': 1.0, 'device_const': {'Nursery_AC_Socket_1A_No_1': (1200000, False), 'Nursery_AC_Socket_1A_No_2': (1200000, False), 'Nursery_AC_Socket_1B': (1200000, False), 'Nursery_AC_Socket_1C': (1200000, False)}, 'timestamp': datetime.datetime(2020, 6, 9, 13, 23, 25, 358279)}, 'nursery2_lights': {'state': 'Unconstrained', 'energy_est_used_total': 0, 'constrainin

In [2]:
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.latest_forecast)

Running Forecasting --  2020-06-09 15:52:51.659354  --
Forecaster : 2020-06-09 15:52:57.900147 : [{"system_load":188.4641548718,"generated_energy":0.0,"consumed_energy":271.0}]


parameter,nursery_1a_cpe_no_1,nursery_1b_cpe_no_3,nursery_1b_cpe_no_4,nursery_1c_cpe_no_5,nursery_1c_cpe_no_6,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_9,nursery_2c_cpe_no_12,nursery_ac_socket_1a_no_1,...,consumed_energy,generated_energy,timestamp,system_load,victron_venusgx,playground_no_1,playground_no_4,nursery_1a_cpe_no_2,nursery_2b_cpe_no_10,nursery_2c_cpe_no_11
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-09 07:00:00,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,1.000000,...,268.000000,33.500000,2020-06-09 07:00:00,184.528500,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 08:00:00,4.850000,5.872500,0.0,3.019500,0.0,2.484500,0.0,1.538500,2.028000,1.500000,...,271.000000,0.000000,2020-06-09 08:00:00,188.464155,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 09:00:00,3.548500,4.695500,0.0,4.647500,0.0,5.154000,0.0,4.201000,1.896000,0.000000,...,280.000000,0.000000,2020-06-09 09:00:00,198.183860,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 10:00:00,1.394214,4.950000,0.0,4.396000,0.0,1.821047,0.0,7.219000,5.408000,0.000000,...,259.000000,0.000000,2020-06-09 10:00:00,187.020764,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 11:00:00,3.640000,7.932000,0.0,1.852372,0.0,4.816000,0.0,4.239000,5.496000,0.000000,...,262.000000,0.000000,2020-06-09 11:00:00,200.979948,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 12:00:00,3.413500,2.868000,0.0,2.898000,0.0,4.741000,0.0,4.672000,5.200667,0.000000,...,256.500000,0.000000,2020-06-09 12:00:00,179.014155,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 13:00:00,2.767000,4.804667,0.0,5.163500,0.0,4.730000,0.0,4.573500,8.019000,0.000000,...,256.500000,0.000000,2020-06-09 13:00:00,170.737822,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 14:00:00,3.827000,3.781000,0.0,3.767500,0.0,4.824000,0.0,3.271500,4.811500,0.000000,...,238.000000,0.000000,2020-06-09 14:00:00,172.968822,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-09 15:00:00,2.515000,4.289000,0.0,5.911000,0.0,4.903000,0.0,7.246000,6.645000,0.000000,...,188.000000,0.000000,2020-06-09 15:00:00,131.613688,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
##Forecaster V2 - ML and PV Models
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster_v2 import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.full_df)

Running Forecasting --  2020-06-09 16:43:50.638295  --
Doing forecast for Dev : consumed_energy
Doing forecast for Dev : system_load
Doing forecast for Dev : streetlight_no_1
Doing forecast for Dev : streetlight_no_2
Doing forecast for Dev : streetlight_no_3
Doing forecast for Dev : playground_no_1
Doing forecast for Dev : playground_no_2
Doing forecast for Dev : playground_no_3
Doing forecast for Dev : playground_no_4
Doing forecast for Dev : playground_no_5
Doing forecast for Dev : playground_ac_socket_no_1
Doing forecast for Dev : playground_ac_socket_no_2
Doing forecast for Dev : nursery_ac_socket_1a_no_1
Doing forecast for Dev : nursery_ac_socket_1a_no_2
Doing forecast for Dev : nursery_ac_socket_1b
Doing forecast for Dev : nursery_ac_socket_1c
Doing forecast for Dev : nursery_1a_cpe_no_1
Doing forecast for Dev : nursery_1a_cpe_no_2
Doing forecast for Dev : nursery_1b_cpe_no_3
Doing forecast for Dev : nursery_1b_cpe_no_4
Doing forecast for Dev : nursery_1c_cpe_no_5
Doing forecast 

,consumed_energy,timestamp,system_load,streetlight_no_1,streetlight_no_2,streetlight_no_3,playground_no_1,playground_no_2,playground_no_3,playground_no_4,...,nursery_ac_socket_2c,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_9,nursery_2b_cpe_no_10,nursery_2c_cpe_no_11,nursery_2c_cpe_no_12,generated_energy,charged_energy,battery_soc
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-09 17:00:00,146.104632,2020-06-09 17:00:00,154.959766,0.121604,0.322975,-0.025512,0.0,0.154478,0.146610,0.0,...,0.217433,0.189541,0.0,0.150473,0.0,0.0,-0.036856,0.000000,-146.104632,40
2020-06-09 18:00:00,139.666555,2020-06-09 18:00:00,151.589273,0.354696,0.369701,0.131983,0.0,0.301292,0.281089,0.0,...,0.412722,0.264362,0.0,0.630446,0.0,0.0,0.292939,0.000000,-139.666555,40
2020-06-09 19:00:00,146.053777,2020-06-09 19:00:00,149.915883,0.568375,0.323218,0.575498,0.0,0.365550,0.299198,0.0,...,0.588920,0.279054,0.0,0.545297,0.0,0.0,0.551567,0.000000,-146.053777,40
2020-06-09 20:00:00,159.394843,2020-06-09 20:00:00,147.571594,0.747944,0.273247,0.140840,0.0,0.390798,0.454911,0.0,...,0.748659,0.304388,0.0,0.556739,0.0,0.0,0.574049,0.000000,-159.394843,40
2020-06-09 21:00:00,176.756764,2020-06-09 21:00:00,145.725408,1.102138,0.219987,0.377886,0.0,0.380417,0.449575,0.0,...,0.894207,0.328913,0.0,0.420595,0.0,0.0,0.865957,0.000000,-176.756764,40
2020-06-09 22:00:00,152.497583,2020-06-09 22:00:00,140.518497,1.155029,0.163624,0.341376,0.0,0.538741,0.202510,0.0,...,1.027521,0.343391,0.0,0.357835,0.0,0.0,0.610819,0.000000,-152.497583,40
2020-06-09 23:00:00,69.953702,2020-06-09 23:00:00,130.172927,0.121386,0.104335,0.312062,0.0,0.437459,-0.046425,0.0,...,1.150288,0.492539,0.0,0.540435,0.0,0.0,0.657334,0.000000,-69.953702,40
2020-06-10 00:00:00,90.206879,2020-06-10 00:00:00,130.069630,0.212317,0.042284,0.304563,0.0,0.346093,0.012415,0.0,...,1.263961,0.679764,0.0,-0.025995,0.0,0.0,0.485789,0.000000,-90.206879,40
2020-06-10 01:00:00,104.287949,2020-06-10 01:00:00,128.805323,0.292196,-0.022370,0.291903,0.0,0.247539,0.056333,0.0,...,1.369794,0.436702,0.0,0.490277,0.0,0.0,-0.095522,0.000000,-104.287949,40


In [6]:
##Remote Logger Example

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Remote_Logger import RemoteLogger

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

mqtt_address = json_data["mqtt_address"]
mqtt_port = json_data["mqtt_port"]
mqtt_username = json_data["mqtt_username"]
mqtt_password = json_data["mqtt_password"]

data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

rl = RemoteLogger(data, mqtt_address, mqtt_port, mqtt_username, mqtt_password)

rl.do_step()
print(rl.getLatest())

Remote Logger: 2020-06-09 16:56:10.376730 : Publishing message with Nulls:['hist_playground_no_1', 'hist_playground_no_4', 'hist_nursery_1a_cpe_no_2', 'hist_nursery_2b_cpe_no_10', 'hist_nursery_2c_cpe_no_11', 'quota2_nursery_ac_socket_1a_no_1', 'quota2_nursery_ac_socket_1a_no_2', 'quota2_nursery_ac_socket_1b', 'quota2_nursery_ac_socket_1c', 'quota2_nursery_ac_socket_2a_no_1', 'quota2_nursery_ac_socket_2a_no_2', 'quota2_nursery_ac_socket_2b', 'quota2_nursery_ac_socket_2c', 'quota2_playground_ac_socket_no_1', 'quota2_playground_ac_socket_no_2']  with return:(0, 3) - Success with Mid: 2 at 2020-06-09 16:56:10.378230

